# Learning SQL using sqlite3 on Linux

This was starting during the Galvanize Data Science prep course. I expect to keep updating this as I learn new things about SQL. Here is the SQLite [documentation](https://sqlite.org/docs.html)

## Opening sqlite session
You can create a blank database file from BASH with `touch <name>.db` command. Next you can connect to the database with `sqlite3 <name>.db`. It is usually best to be in the same directory as the database file is located.

## SQLite commands

+ `.tables` - shows a list of tables in the database
+ `.read filename.sql` - runs the contents of as file as a SQL script
+ `.headers ON` - will display headers for data returned ([make permanent](https://stackoverflow.com/questions/5924149/how-to-configure-sqlite-to-display-headers-by-default))
+ `.mode column` - improves readability by aligning text and columns

## Creating tables
Building the `sales.db` database by adding in some tables and content.
```sqlite
CREATE TABLE customers (
id INTEGER PRIMARY KEY,
name VARCHAR(50),
birthyear INTEGER,
city VARCHAR(50),
state VARCHAR(2));

CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name VARCHAR(50),
    price NUMERIC(6, 2));

CREATE TABLE purchases (
    id INTEGER PRIMARY KEY,
    custid INTEGER REFERENCES customers(id),
    prodid INTEGER REFERENCES products(id),
    quantity INTEGER,
    date DATE);
```

## Manually entering data

```sqlite
INSERT INTO customers VALUES
    (1001, 'Polly', 1952, 'San Francisco', 'CA');

INSERT INTO customers (id, name, city, state) VALUES
  (1002, 'Juan', 'Denver', 'CO');

INSERT INTO customers VALUES
  (1003, 'Chiron', 1980, 'Seattle', 'WA'),
  (1004, 'Petra', 1992, 'New York', 'NY'),
  (1005, 'Arvind', 1974, 'Phoenix', 'AZ');

INSERT INTO customers (id, name, city, state) VALUES
  (1006, 'Pei', 'Austin', 'TX'),
  (1007, 'Pira', 1977, 'CA');
```

### Removing rows
```sqlite
DELETE FROM customers
  WHERE id=1007;
```

## Writing Queries

Order of Components | Order of| Evaluation $\phantom{ssssssssssssssssssssssssssssssssssssssssssssss}$
 --- | ---
 `SELECT` | 5 - | Targeted list of columns evaluated and returned
 `FROM` | 1 - | Product of all tables is formed
 `JOIN` . `ON` . `AND` | 1 - | "            " and rows filtered out from joined table that do not meet criteria
 `WHERE` | 2 - | Rows filtered out that do not meet criteria
 `GROUP BY` | 3 - | Rows combined using `GROUP BY` clause and aggregations applied
 `HAVING` | 4 - | Aggregations that do not meet `HAVING` criteria are removed
 `ORDER BY` | 6 - | Rows sorted by column(s)
 `LIMIT` | 7 - | Final table truncated based on limit size
 `;` | 8 - | Semicolon ends the query
 

### Simple queries
Starting off using `sales.db` for examples.
```sqlite
SELECT * FROM customers;

SELECT id, name FROM customers;

SELECT * FROM customers WHERE state = 'CA';

SELECT * FROM products WHERE price > 10.00;

SELECT * FROM purchases
  WHERE date BETWEEN '2017-02-01' AND '2017-02-28';

SELECT id, custid, prodid, date
FROM purchases
WHERE custid IN (1001, 1003, 1005);

SELECT name, state FROM customers
  WHERE name LIKE 'P%';

SELECT name, state FROM customers
  WHERE name LIKE '__i%';

SELECT * FROM tablename LIMIT 5;

PRAGMA table_info(tablename);
```

### String functions
```sqlite
SELECT id, name FROM products
  WHERE LOWER(name) LIKE '%razor%';

SELECT REPLACE(name, 'n', 'x') FROM customers;
```
Find a fuller list of SQLite core functions [here](https://sqlite.org/lang_corefunc.html)

### Order by
```sqlite
SELECT name, city, state
FROM customers
ORDER BY state;

SELECT id, name
FROM products
ORDER BY price;

SELECT name, price
FROM products
ORDER BY price DESC;
```

### Case when
Helpful for creating $1$ or $0$ in data or dealing with `NULL` values in tables, see StackOverflow [answer to creating bools](https://stackoverflow.com/questions/157114/how-to-output-a-boolean-in-t-sql-based-on-the-content-of-a-column)
```sqlite
SELECT 
    name,
    city,
    CASE WHEN state = 'CO' THEN 1 ELSE 0 END AS in_state
FROM
    customers
;
```


### Aggregate queries
Using `world.db` for these examples.
```sqlite
SELECT continent
FROM country
GROUP BY continent;
```
However, if you try to return something that has multiple answers it can return an error.
```sqlite
SELECT name, continent
FROM country
GROUP BY continent;
```
Here there would be multiple names to return for each continent group. Instead you could use an aggregate function like `COUNT`, `SUM`, `AVG`, `MAX`, `MIN` to perform an operation on the data.
```sqlite
SELECT COUNT(name), continent
FROM country
GROUP BY continent;
```
Functions can be used just to get a single answer about the data.
```sqlite
SELECT COUNT(*) as big_countries
FROM country
WHERE population > 100000000;

SELECT SUM(population) as the_whole_world
FROM country;
```
To eliminate repeats `DISTINCT` is used. Assume `DISTINCT` counts for all fields in `SELECT` clause.
```sqlite
SELECT DISTINCT continent, governmentform
FROM country
ORDER BY continent, governmentform;
```
Say lets find the country in each continent that has the largest area.
```sqlite
SELECT continent, name, MAX(surfacearea) FROM country
GROUP BY continent;
```

Aggregators cannot be used in `WHERE` clause. `WHERE` clause filters rows in root table *before* aggregation. Use the `HAVING` clause instead. Like `WHERE`, `HAVING` cannot reference an alias (in Postgres, at least).
```sqlite
SELECT 
    continent, 
    COUNT(*) AS num_countries
FROM
    country
-- WHERE
--     COUNT(*) > 2
GROUP BY
    continent
HAVING
    COUNT(*) > 2
```

## Joining Tables
*Using the* `sales.db` *for examples.*
There are 3 tables of which the `purchases` table has a reference to both of the other tables and is the only one which references the other tables. This is known as the fact table, it has foreign keys that reference other tables. The dimension tables branch off the central fact table.

To join tables we use the `JOIN` and `ON` keywords to specify the table to join and on which fields they should be linked.
```sqlite
SELECT name, city, state, purchases.id, date
FROM customers
JOIN purchases
ON customer.id = purchases.custid;
```
You can also make aliases for the tables like so,
```sqlite
SELECT cu.name, cu.city, cu.state, pu.id, pu.date
FROM customers cu
JOIN purchases pu
ON cu.id = pu.custid;
```

### Types of joins
There are 3 types of joins: inner, full outer, and left or right outer. The previous join was an 'inner' join. A `LEFT JOIN` simply uses all the records from the `FROM tablename` table and leaves blank the fields from the joined table if a record doesn't exist. A `RIGHT JOIN` does the same thing but for the `JOIN tablename` table. A `FULL JOIN` includes all entries from both tables.
```sqlite
SELECT cu.name, cu.city, cu.state, pu.id, pu.date
FROM customers cu
LEFT JOIN purchases pu
ON cu.id = pu.custid;
```

### Multiple joins
Multiple joins can be created by chaining `JOIN tablename ON first.id = second.id` statements.
```sqlite
SELECT cu.name as customer,
       cu.city,
       cu.state,
       pu.id as purchase_id,
       pu.date,
       pr.name as product_name,
       pr.id as product_id
FROM customers cu
JOIN purchases pu
ON cu.id = pu.custid
JOIN products pr
ON pu.prodid = pr.id;
```

### Conditional joins
Conditional logic can be applied to what entries make it into the join. The keyword `AND` is used to do this. This is necessary to preserve `LEFT JOIN` when a condition is applied to the joining table. [Read this answer](https://stackoverflow.com/questions/1018952/condition-within-join-or-where) on StackOverflow for more insight.
```sqlite
-- how many of each meal type were bought
SELECT
    ml.type,
    COUNT(*)
FROM
    meals AS ml
JOIN
    events AS ev
  ON
    ml.meal_id = ev.meal_id
  AND
    ev.event = 'bought'
GROUP BY
    ml.type
;
```

### Subqueries
- In general, you can replace and table name with a subquery
```sqlite
SELECT ... FROM (SELECT ...) ...
```
- If query returns a single value, you can use it as a value
```sqlite
... WHERE col1 = (SELECT ...) ...
```
- If query returns a single column, you can use it as a vector
```sqlite
... WHERE col1 IN (SELECT ...) ...
```
example:
```sqlite
SELECT
    newTable.col1,
    newTable.col2
FROM
    (SELECT
        col1,
        col2,
        col3
     FROM
        anotherTable) AS newTable
;
```

#### Temporary tables
A single temporary table can be used in place of multiple identical subqueries. Temp tables are held in memory (if small enough) just like subqueries are and then dropped at the end of the session.
```sqlite
WITH newTable AS
    (SELECT
         col1,
         col2,
         col3
      FROM
         anotherTable)
-- can also have more than one temp table
-- within WITH clause

SELECT
    newTable.col1,
    newTable.col2
FROM
    newTable
;
```

#### Create/Drop table
This creates a new table from a query that can be used between sessions until dropped. __*BE CAREFUL OF DROPPING TABLES*__ they cannot be recovered and would need to be completely re-inserted if the data even still exists.
```sqlite
CREATE TABLE newTable AS
    (SELECT
         col1,
         col2,
         col3
      FROM
         anotherTable);

SELECT
    newTable.col1,
    newTable.col2
FROM
    newTable
;
```


## SQL style conventions
1. Use CAPS for SQL keywords
2. Keep lines under 80 characters
3. If a query is really short, you can put it on one line
4. If query is running long, or becoming hard to parse, break it down
  * Break query up so keywords begin on their own line
    ```sqlite
    SELECT name
    FROM mytable
    WHERE price > 100
    ORDER BY name;
    ```
  * Put each column of `SELECT` on its own line, *indented*
    ```sqlite
    SELECT
        name,
        SUM(price) as total_price,
        COUNT(1)
    FROM mytable
    GROUP BY name;
    ```
5. Nested queries can get tricky, best to keep everything in the nested query on the same indentation level.

## Additional Resources
[PostgreSQL: Documentation: 8.2: Aggregate Functions](https://www.postgresql.org/docs/8.2/functions-aggregate.html)

[SQLite documentation](https://sqlite.org/docs.html)

[W3Schools SQL Tutorial](https://www.w3schools.com/sql/default.asp)

[SQLZoo Tutorial](http://sqlzoo.net/)

[SQL challenges](https://www.hackerrank.com/domains/sql) on hackerrank